#### Google Palm LLM & API key setup

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase
from langchain.agents.agent_types import AgentType

c:\Users\N\Desktop\Tunga.io\capstone\version2\hrstreamline\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Your API key
api_key = ''

# Use the correct model name
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=api_key,
    temperature=0.2
)

#### Connect with database and ask some basic questions

In [3]:
# Relative path to the SQLite DB file
db_path = "../../instance/hr_streamline_app.db"

# Connect using LangChain
db = SQLDatabase.from_uri(f"sqlite:///{db_path}", sample_rows_in_table_info=3)

# View table info
print("Database schema:")
print(db.table_info)


Database schema:

CREATE TABLE alembic_version (
	version_num VARCHAR(32) NOT NULL, 
	CONSTRAINT alembic_version_pkc PRIMARY KEY (version_num)
)

/*
3 rows from alembic_version table:
version_num
61b9efb95ef4
*/


CREATE TABLE attendance (
	id INTEGER NOT NULL, 
	employee_id INTEGER NOT NULL, 
	date DATE NOT NULL, 
	clock_in_time DATETIME, 
	clock_out_time DATETIME, 
	total_hours NUMERIC(4, 2) NOT NULL, 
	status VARCHAR(8) NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(employee_id) REFERENCES employee (id) ON DELETE CASCADE, 
	CONSTRAINT unique_employee_date UNIQUE (employee_id, date)
)

/*
3 rows from attendance table:
id	employee_id	date	clock_in_time	clock_out_time	total_hours	status
1	1	2025-06-18	2025-06-18 01:41:46	None	0.00	Present
2	2	2025-06-18	2025-06-18 01:41:46	None	0.00	Present
3	3	2025-06-18	2025-06-18 01:41:46	None	0.00	Present
*/


CREATE TABLE auth (
	id INTEGER NOT NULL, 
	email VARCHAR(50) NOT NULL, 
	password_hash VARCHAR(255) NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (em

In [4]:
# Create SQL agent (better than SQLDatabaseChain)
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False ,  # Set to False for cleaner output
    handle_parsing_errors=True
)

In [5]:
# Execute your query
try:
    result1 = agent_executor.invoke("How many male employees do we have?")
    print("Answer:", result1['output'])
except Exception as e:
    print(f"Error: {e}")
    # Try a more specific query
    result1 = agent_executor.invoke("Count the total number of records in the employee table")
    print("Answer:", result1['output'])

Answer: There are 2 male employees.


In [6]:
result1

{'input': 'How many male employees do we have?',
 'output': 'There are 2 male employees.'}

Above is the correct answer 👍🏼

In [7]:
try:
    result2 = agent_executor.invoke("Which employee has an annual leave request starting soon?")
    print("Answer:", result2['output'])
except Exception as e:
    print(f"Error: {e}")
    # More specific fallback
    result2 = agent_executor.invoke(
        "List the employee names and start dates for pending annual leave requests ordered by start date."
    )
    print("Answer:", result2['output'])


Answer: Cristiano Ronaldo has an annual leave request starting on 2025-07-01.


In [8]:
result2

{'input': 'Which employee has an annual leave request starting soon?',
 'output': 'Cristiano Ronaldo has an annual leave request starting on 2025-07-01.'}

In [9]:
try:
    result3 = agent_executor.invoke("if cristiano goes on leave how many people will be left in his department and whats their position in the company?")
    print("Answer:", result3['output'])
except Exception as e:
    print(f"Error: {e}")
    # More specific fallback
    result3 = agent_executor.invoke(
        "Get the department of the employee named Cristiano. Then count how many employees are in that department excluding Cristiano, and list their positions and ranks."
    )
    print("Answer:", result3['output'])


Answer: There will be 1 person left in Cristiano's department, and their position is Senior Software Engineer.


In [10]:
result3

{'input': 'if cristiano goes on leave how many people will be left in his department and whats their position in the company?',
 'output': "There will be 1 person left in Cristiano's department, and their position is Senior Software Engineer."}

In [11]:
try:
    result4 = agent_executor.invoke("if everyone in cristiano department goes on leave for a week how many days of leave will each person have left?")
    print("Answer:", result4['output'])
except Exception as e:
    print(f"Error: {e}")
    # More specific fallback with a simpler or safer prompt
    result4 = agent_executor.invoke(
    "If every employee in the Cristiano department takes 5 days of leave, how many leave days will each person remaining?"
    )
print("Answer:", result4['output'])


Answer: The remaining leave balance for employees in the Engineering department after taking a week of leave is 3 and 5 days.
Answer: The remaining leave balance for employees in the Engineering department after taking a week of leave is 3 and 5 days.


In [12]:
result4

{'input': 'if everyone in cristiano department goes on leave for a week how many days of leave will each person have left?',
 'output': 'The remaining leave balance for employees in the Engineering department after taking a week of leave is 3 and 5 days.'}

In [14]:
try:
    result5 = agent_executor.invoke("Show the most recent employee who requested leave with pending or approved status, are they currently active, when does their leave end, how long are they staying and their positions.")
    print("Answer:", result5['output'])
except Exception as e:
    print(f"Error: {e}")
    # Attempt structured fallback
    try:
        result5 = agent_executor.invoke(
            "List all employees who have requested leave with pending or approved status. Include whether they are currently active, their expected end dates, leave duration and their job positions."
        )
        print("Answer:", result5['output'])
    except Exception as inner_e:
        print("Still failed:", inner_e)



Answer: The most recent employee who requested leave with pending or approved status is currently active, their leave ends on 2025-07-05, they are staying for 5 days, and their position is Software Engineer.


In [15]:
result5

{'input': 'Show the most recent employee who requested leave with pending or approved status, are they currently active, when does their leave end, how long are they staying and their positions.',
 'output': 'The most recent employee who requested leave with pending or approved status is currently active, their leave ends on 2025-07-05, they are staying for 5 days, and their position is Software Engineer.'}

#### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [16]:
few_shots = [
    {
        'Question': "How many male employees do we have?",
        'SQLQuery': "SELECT COUNT(*) FROM employee WHERE gender = 'Male';",
        'SQLResult': "Result of the SQL query",
        'Answer': result1['output']
    },
    {
        'Question': "Which employee has an annual leave request starting soon?",
        'SQLQuery': """
            SELECT 
                employee.first_name,      
                employee.last_name,       
                leave_requests.start_date 
            FROM 
                leave_requests
            JOIN 
                employee ON leave_requests.employee_id = employee.id 
            WHERE 
                leave_requests.leave_type = 'Annual' 
                AND leave_requests.status = 'Pending'
            ORDER BY 
                leave_requests.start_date ASC
            LIMIT 1;
        """,
        'SQLResult': "Result of the SQL query",
        'Answer': result2['output']
    },
    {
        'Question': "If Cristiano goes on leave, how many people will be left in his department and what are their positions?",
        'SQLQuery': """
            SELECT 
                emp_position,
                COUNT(*) AS number_of_employees,
                (SELECT COUNT(*) 
                 FROM employee 
                 WHERE emp_department = e.emp_department 
                   AND first_name <> 'Cristiano') AS total_employees_left
            FROM 
                employee e
            WHERE 
                e.emp_department = (SELECT emp_department FROM employee WHERE first_name = 'Cristiano' LIMIT 1)
                AND e.first_name <> 'Cristiano'
            GROUP BY 
                emp_position;
        """,
        'SQLResult': "Result of the SQL query",
        'Answer': result3['output']
    },
    {
        'Question': "If everyone in Cristiano department goes on leave for a week, how many days of leave will they have left?",
        'SQLQuery': """
            SELECT 
                first_name,
                last_name,
                CASE 
                    WHEN emp_leave_balance - 7 < 0 THEN 0
                    ELSE emp_leave_balance - 7
                END AS leave_balance_after_week
            FROM 
                employee
            WHERE 
                emp_department = (SELECT emp_department FROM employee WHERE first_name = 'Cristiano' LIMIT 1);
        """,
        'SQLResult': "Result of the SQL query",
        'Answer': result4['output']
    },
    {
        'Question': (
            "Show the most recent employee who requested leave with pending or approved status, "
            "are they currently active, when do they plan to return from leave, how long are they staying, and their positions."
        ),
        'SQLQuery': """
            SELECT 
                e.first_name, 
                e.last_name, 
                e.emp_status, 
                l.end_date, 
                l.days_requested, 
                e.emp_position 
            FROM 
                employee e 
            JOIN 
                leave_requests l ON e.id = l.employee_id 
            WHERE 
                l.status IN ('Pending', 'Approved') 
            LIMIT 10;
        """,
        'SQLResult': "Result of the SQL query",
        'Answer': result5['output']
    }
]


In [13]:
few_shots[0]["SQLQuery"]

"SELECT COUNT(*) FROM employee WHERE gender = 'Male';"

In [17]:
result5['output']

'Cristiano Ronaldo is an active employee with the position of Software Engineer. He requested 5 days of leave and his leave request is either pending or approved. His expected return date is not specified.'

### Creating Semantic Similarity Based example selector

- create embedding on the few_shots
- Store the embeddings in Chroma DB
- Retrieve the the top most Semantically close example from the vector store

In [17]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\N\AppData\Local\Temp\ipykernel_10668\3938620714.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [18]:
to_vectorize = [" ".join(example.values()) for example in few_shots]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

In [19]:
to_vectorize

["How many male employees do we have? SELECT COUNT(*) FROM employee WHERE gender = 'Male'; Result of the SQL query There are 2 male employees.",
 "Which employee has an annual leave request starting soon? \n            SELECT \n                employee.first_name,      \n                employee.last_name,       \n                leave_requests.start_date \n            FROM \n                leave_requests\n            JOIN \n                employee ON leave_requests.employee_id = employee.id \n            WHERE \n                leave_requests.leave_type = 'Annual' \n                AND leave_requests.status = 'Pending'\n            ORDER BY \n                leave_requests.start_date ASC\n            LIMIT 1;\n         Result of the SQL query Cristiano Ronaldo has an annual leave request starting on 2025-07-01.",
 "If Cristiano goes on leave, how many people will be left in his department and what are their positions? \n            SELECT \n                emp_position,\n           

In [20]:
sqlite_prompt = """You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here
"""


In [22]:
from langchain.prompts.prompt import PromptTemplate


example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer"],
    template="""
Question: {Question}
SQLQuery: {SQLQuery}
SQLResult: {SQLResult}
Answer: {Answer}
""".strip()
)

In [23]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _sqlite_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


### Setting up PromptTemplate using input variables

In [158]:
print(_sqlite_prompt)

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: 

In [24]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=sqlite_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"],
)

In [26]:
from langchain_experimental.sql import SQLDatabaseChain


new_chain = SQLDatabaseChain.from_llm(
    llm=llm,
    db=db,
    prompt=few_shot_prompt,
    verbose=True,
)


In [29]:
result = new_chain.invoke({"query": "How many female employees do we have?"})
print("Answer:", result)




> Entering new SQLDatabaseChain chain...
How many female employees do we have?
SQLQuery:```sqlite
SELECT COUNT(*) FROM employee WHERE gender = 'Female'
```

OperationalError: (sqlite3.OperationalError) near "```sqlite
SELECT COUNT(*) FROM employee WHERE gender = 'Female'
```": syntax error
[SQL: ```sqlite
SELECT COUNT(*) FROM employee WHERE gender = 'Female'
```]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [95]:
print(new_chain.input_keys)


['query']


Now this is working ok. Previously for this same question it was giving wrong answer because it did not use SUM clause around stock_quantity column

In [ ]:
new_chain("How much is the price of the inventory for all small size t-shirts?")

In [ ]:
new_chain("How much is the price of all white color levi t shirts?")

In [ ]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")

In [ ]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")

In [ ]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')